<a href="https://colab.research.google.com/github/adityakalra/pytrader/blob/master/Tesla_PricePrediction_365.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PROJECT :  PRICE PREDICTION Using LSTM (Artifical Recurrent Neural Network)
PRICE PREDICTION --> Stock Market
SOURCE --> Yahoo
ENVIORNMENT --> COLABS, Keras , Skylearn , Math, Pandas, Matplotlib, Numpy , 


In [0]:
# import liabraries
import datetime as dt
import math
import numpy as np  
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import pandas_datareader as web
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense , LSTM

In [0]:
dat = dt.date.today()-datetime.timedelta(1)
print(dat)

In [0]:
# Create DataFrame & get Stock Quote
df = web.DataReader ('TSLA', data_source='yahoo', start='2012-01-01',end= dat.strftime('%Y-%m-%d'))
#show the DataFrame
df

In [0]:
# Get No. of Rows & Columns in the data set
df.shape

In [0]:
# Visualize the closing price History in Matplotlib
plt.figure(figsize=(18,6))
plt.title('TESLA - Closing Price')
plt.plot(df['Close'])
plt.xlabel('Date',fontsize=18)
plt.ylabel('Closing Price USD',fontsize=18)
plt.show()


In [0]:
# Create new DataFrame only using the close column
data = df.filter(['Close'])
#Create the DataFrame to numpy array
dataset = data.values
#Get the number of rows to train the model on 
training_data_len = math.ceil(len(dataset)* 0.8)

training_data_len

In [0]:
# Scaling of DataFram
scaler = MinMaxScaler( feature_range=(0,1))
scaler_data = scaler.fit_transform(dataset)
scaler_data

In [0]:
# create the training data set
# Create the scaled training Data
train_data = scaler_data[0:training_data_len, :]
# Split the data  into x_train and y_train data sets
x_train =[]
y_train =[]

for i in range (365, len(train_data)):
  x_train.append(train_data[i-365:i,0])
  y_train.append(train_data[i,0])
  if i<= 366:
    print(x_train)
    print(y_train)
    print()

In [0]:
#convert the x_train and y_train to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)

In [0]:
#Reshape the data for LSTM
x_train = np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))
x_train.shape

In [0]:
# Build the LSTM MODEL
model = Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(x_train.shape[1], 1)))
model.add(LSTM(50,return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

In [0]:
#Compile the Model
model.compile(optimizer='adam',loss='mean_squared_error')

In [0]:
#Train the Model
model.fit(x_train,y_train,batch_size=1,epochs=1)


In [0]:
#Create the testing Data set
# Create a new array containing scaled values from index 1543 to 2003
test_data =scaler_data[training_data_len -365:, :]
#Create the data sets x_test and y_test
x_test = []
y_test = dataset[training_data_len: , :]

for i in range (365,len(test_data)):
  x_test.append(test_data[i-365:i,0])

In [0]:
# Convert the data to a numpy Array
x_test = np.array(x_test)

In [0]:
# Reshape the data
x_test = np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))

In [0]:
x_test

In [0]:
from keras.models import Sequential
from keras.layers import Dense , LSTM
# Get the models Prediction price values
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

In [0]:
# Get the root mean squared error [RMSE] is standard deviation of the residuals 
rmse = np.sqrt(np.mean(predictions-y_test)**2)
rmse

In [0]:
# Plot the Data
train = data [:training_data_len]
valid = data [training_data_len:]
valid['Predictions'] = predictions
#I also want this data to be exported in excel
# valid.to_excel('C:\\data.xlsx')

# this works in normal python

In [0]:
#Show the valid and predicted price
valid

In [0]:
#Visualize the data
plt.figure(figsize=(17,6))
plt.title('Model')
plt.xlabel('Date',fontsize=18)
plt.ylabel('Close Price',fontsize=18)
plt.plot(train['Close'])
plt.plot(valid[['Close','Predictions']])
plt.legend(['Train','Val','Predictions'],loc='lower right')
plt.show()